In [ ]:
from selenium import webdriver as wd
from bs4 import BeautifulSoup as bs
import requests
import datetime
from datetime import date
import pandas as pd
import time
import os, sys

In [ ]:
spx_url = 'https://kr.investing.com/indices/us-spx-500-historical-data'
dji_url = 'https://kr.investing.com/indices/us-30-historical-data'
nas_url = 'https://kr.investing.com/indices/nasdaq-composite-historical-data'
kospi_url = 'https://kr.investing.com/indices/kospi-historical-data'
kosdaq_url = 'https://kr.investing.com/indices/kosdaq-historical-data'
sox_url = 'https://kr.investing.com/indices/phlx-semiconductor-historical-data'
vix_url = 'https://kr.investing.com/indices/volatility-s-p-500-historical-data'

urls = [spx_url, dji_url, nas_url, kospi_url, kosdaq_url, sox_url, vix_url]

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) \
           AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

In [ ]:
column_eng = ['date', 'price', 'open', 'high', 'low', 'volume', 'change(%)']
column_kor = ['날짜', '종가', '시가', '고가', '저가', '거래량', '변동(%)']
f_name = ['s&p', 'dji', 'nasdaq', 'kospi', 'kosdaq', 'sox', 'vix' ]

In [ ]:
dir_name = 'data/indices_major/'
start_ix = 'aa'

In [ ]:
today = datetime.date.today()
today = today.strftime('%Y%m%d')

In [ ]:
for i, url in enumerate(urls):
    res = requests.get(url, headers=headers)
    df = pd.read_html(res.text, attrs={"id": "curr_table"}, flavor=["lxml", "bs4"])[0]
    df.columns = column_eng
    df['date'] = df['date'].apply(lambda x : datetime.datetime.strptime(x, "%Y년 %m월 %d일"))
    filename = '{}{}{}_{}_{}.csv'.format(dir_name, start_ix,i+1,f_name[i], today)
    df.to_csv(filename, index=False)

#### get cpi

In [ ]:
cpi_url = 'https://kr.investing.com/economic-calendar/cpi-733'

In [ ]:
res = requests.get(cpi_url, headers=headers)
df = pd.read_html(res.text)[0]

In [ ]:
df.columns=  ['release_date', 'time', 'acrual_rate', 'forecast_rate', 'prev_rate', 'none']
df['release_date'] = df['release_date'].\
                    apply(lambda x : datetime.datetime.strptime(x[:13], "%Y년 %m월 %d일"))
df['time'] = df['time'].apply(lambda x : datetime.datetime.strptime(x, "%H:%M").time())

In [ ]:
df.head()

In [ ]:
f_name = 'cpi_index'
i = 7
filename = '{}{}{}_{}_{}.csv'.format(dir_name, start_ix,i+1, f_name, today)
df.to_csv(filename, index=False)

#### get fear and greed

In [ ]:
import pytz, json

In [ ]:
today = datetime.date.today()
today_p = today.strftime('%Y%m%d')
diff_days = datetime.timedelta(days=30)
today_s = today - diff_days
start_date = today_s.strftime('%Y-%m-%d')

In [ ]:
BASE_URL = "https://production.dataviz.cnn.io/index/fearandgreed/graphdata"
# START_DATE = '2022-01-01'
START_DATE = start_date

In [ ]:
def transform_timezone(x):
    dt = datetime.datetime.fromtimestamp(x / 1000, tz=pytz.utc)
    eastern = pytz.timezone('US/Eastern')
    # eastern = pytz.timezone('Asia/Seoul')
    loc_dt = dt.astimezone(eastern)
    return loc_dt.date()

In [ ]:
r = requests.get("{}/{}".format(BASE_URL, START_DATE), headers=headers)
data = r.json()
fg_data = data['fear_and_greed_historical']['data']

In [ ]:
df = pd.DataFrame(fg_data)
df.columns = ['date', 'f_g_index', 'rating']
# df.head()

In [ ]:
df['date'] = df['date'].apply(lambda x: transform_timezone(x))
df['f_g_index'] = df['f_g_index'].astype('int')
df = df.drop(df.index[-1]) # 마지막 중복 제거 (같은 날 두가지 시간)

In [ ]:
dir_name = 'data/indices_major/'
f_name = 'fear_greed'
i = 8
filename = '{}{}{}_{}_{}.csv'.format(dir_name, start_ix,i+1, f_name, today_p)
df.to_csv(filename, index=False)